## RAG

Aplicação de busca semântica em documentos jurídicos usando Retrieval Augment Generation.

## Setup

In [1]:
import zipfile
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

/home/rwallan/git/langchain-course/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_ = load_dotenv(find_dotenv('.env'))

In [3]:
embedder = OpenAIEmbeddings(model='text-embedding-ada-002')
openai = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)

In [4]:
ROOT = Path().cwd()
ZIP_FILE = ROOT / 'documentos.zip'
EXTRACTED_PATH = ROOT / 'docs'

## Documentos

In [5]:
with zipfile.ZipFile(ZIP_FILE, 'r') as f:
    f.extractall(EXTRACTED_PATH)

In [6]:
loaders = [PyMuPDFLoader(doc) for doc in EXTRACTED_PATH.glob('*.pdf')]
documents = []
for loader in loaders:
    documents.extend(loader.load())

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)
chunks = text_splitter.create_documents(
    [doc.page_content for doc in documents]
)

### Vectorstore

In [8]:
vector_store = PineconeVectorStore.from_documents(
    index_name='llm',
    documents=chunks,
    embedding=embedder,
)

### Search

In [9]:
queries = [
    'Responda com base apenas no input oferecido. Qual foi o número de processo que trata de violação de normas ambientais pela empresa de construção?',
    'Responda com base apenas no input oferecido. Qual foi a decisão no caso de fraude financeira?',
    'Responda com base apenas no input oferecido. Quais foram as alegações no caso de negligência médica?',
    'Responda com base apenas no input oferecido. Quais foram as alegações no caso 822162'
]

In [10]:
retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3},
)

### Usando create_retrieval_chain

In [11]:
combine_docs_chain = create_stuff_documents_chain(
    openai,
    hub.pull('langchain-ai/retrieval-qa-chat')
)
chain = create_retrieval_chain(
    retriever,
    combine_docs_chain
)

/home/rwallan/git/langchain-course/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
responses = [chain.invoke({'input': query}) for query in queries]

In [13]:
for q, r in zip(queries, responses):
    print(f'Query: {q}')
    print(f'Resposta: {r["answer"]}')
    print('---')

Query: Responda com base apenas no input oferecido. Qual foi o número de processo que trata de violação de normas ambientais pela empresa de construção?
Resposta: O número do processo que trata de violação de normas ambientais pela empresa de construção é 175543.
---
Query: Responda com base apenas no input oferecido. Qual foi a decisão no caso de fraude financeira?
Resposta: A decisão no caso de fraude financeira envolvendo João Almeida foi de culpado, com uma sentença de dez anos de prisão e a restituição de $1,000,000 ao Banco Nacional.
---
Query: Responda com base apenas no input oferecido. Quais foram as alegações no caso de negligência médica?
Resposta: As alegações no caso de negligência médica foram de que o Dr. Pedro Sousa não realizou os procedimentos médicos com o cuidado necessário, resultando em complicações graves, incluindo uma cirurgia mal-sucedida que levou a infecções adicionais.
---
Query: Responda com base apenas no input oferecido. Quais foram as alegações no caso 

### Utilizando LCEL

In [14]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [15]:
qa_chain = (
    {
        'context': retriever | format_docs,
        'question': RunnablePassthrough(),
    }
    | hub.pull('rlm/rag-prompt')
    | openai
    | StrOutputParser()
)

/home/rwallan/git/langchain-course/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
responses = [qa_chain.invoke(query) for query in queries]

In [17]:
for q, r in zip(queries, responses):
    print(f'Query: {q}')
    print(f'Resposta: {r}')
    print('---')

Query: Responda com base apenas no input oferecido. Qual foi o número de processo que trata de violação de normas ambientais pela empresa de construção?
Resposta: O número do processo que trata de violação de normas ambientais pela empresa de construção é 175543.
---
Query: Responda com base apenas no input oferecido. Qual foi a decisão no caso de fraude financeira?
Resposta: No caso de fraude financeira envolvendo Carlos Pereira, a decisão foi de culpado, com uma sentença de cinco anos de prisão e uma multa de $200,000.
---
Query: Responda com base apenas no input oferecido. Quais foram as alegações no caso de negligência médica?
Resposta: As alegações no caso de negligência médica foram que Dr. Pedro Sousa não realizou os procedimentos médicos com o cuidado necessário, resultando em complicações graves, incluindo uma cirurgia mal-sucedida que levou a infecções adicionais. Depoimentos de especialistas médicos apoiaram as alegações de negligência, resultando na condenação de Dr. Pedro 